In [68]:
"""
Visualize atom-wise similrity between two alleles
Input: 2 pdb files and 2 DAT files
Output: 2 pdb files with b factor replaced by similarity score
"""

from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.cm as cm
from biopandas.pdb import PandasPdb
from pathlib import Path

In [69]:
class compare():

    def __init__(self, DAT1, DAT2) -> None:
        
        # fixed parameters
        self.l = 10 # charge param
        self.sigma = 0.1 # shape param
        self.d = 0.1 # depth param

        self.DAT1 = DAT1
        self.DAT2 = DAT2

    def CloudSimilarity(self, CoordA, ChargeA, DepthA, CoordB, ChargeB, DepthB):
        """
        adjusted sum of all pair of atoms between two atom clouds
        """
        # spatial only
        # SimScore = np.sum(np.exp(-cdist(CoordA, CoordB)/2*self.sigma*self.sigma))

        # spatial + electrostatic
        # SimScore = np.sum(np.exp(-cdist(ChargeA, ChargeB, "cityblock")**2/self.l) * np.exp(-cdist(CoordA, CoordB)/2*self.sigma*self.sigma))
        # SimScore = np.exp(-cdist(ChargeA, ChargeB, "cityblock")**2/self.l) * np.exp(-cdist(CoordA, CoordB)/2*self.sigma*self.sigma)

        # spatial + electrostatic + depth to groove
        # SimScore = np.exp(np.reciprocal(np.outer(DepthA,DepthB))*self.d) * np.exp(-cdist(ChargeA, ChargeB, "cityblock")**2*self.l) * np.exp(-cdist(CoordA, CoordB, "cityblock")**2/2*self.sigma*self.sigma)
        # SimScore = np.exp(-cdist(ChargeA, ChargeB, "euclidean")**2*self.l) * np.exp(-cdist(CoordA, CoordB, "euclidean")**2*0.5*self.sigma)

        # New kernel: spatial + electrostatic
        SimScore = np.reciprocal(np.cosh(0.5*cdist(CoordA, CoordB, "euclidean"))**1) * np.reciprocal(np.cosh(5*cdist(ChargeA, ChargeB, "euclidean"))**1)
        # SimScore = np.reciprocal(np.cosh(0.5*cdist(CoordA, CoordB, "euclidean"))**1) * np.reciprocal(np.cosh(5*cdist(ChargeA, ChargeB, "euclidean"))**1) * np.exp(-0.05*np.outer(DepthA,DepthB)**2)
        # SimScore = np.reciprocal(np.cosh(0.5*cdist(CoordA, CoordB, "euclidean"))**1) * np.reciprocal(np.cosh(5*cdist(ChargeA, ChargeB, "euclidean"))**1) * np.reciprocal(1+10*np.exp(np.outer(DepthA,DepthB) - 9))

        return SimScore

    def ParamExtract(self, DATFile):
        DAT = pd.read_csv(DATFile)

        resnum = DAT['ResNum'].values
        coord = DAT[['X', 'Y', 'Z']].values
        charge = DAT['Charge'].values # need to be 2D for cdist to work
        depth = DAT['Depth'].values
        in_groove = DAT['InGroove'].values

        return resnum, coord, charge, depth, in_groove

    def color_by_depth(self):
        self.CoordA, _, self.score1, _ = self.ParamExtract(self.DAT1)
        self.CoordB, _, self.score2, _ = self.ParamExtract(self.DAT2)

        self.score1 = self.score1.reshape(-1)
        self.score2 = self.score2.reshape(-1)

        return

    def color_by_InGroove(self):
        self.CoordA, _, _, self.score1 = self.ParamExtract(self.DAT1)
        self.CoordB, _, _, self.score2 = self.ParamExtract(self.DAT2)

        self.score1 = self.score1.reshape(-1)
        self.score2 = self.score2.reshape(-1)

        return

    def filter(self):
        """
        
        """
        return

    def CalcDist(self, depth_cut=0, contact=None, only_groove=0):
        
        """
        Distance of two molecules in PDB files
        """

        resnumA, self.CoordA, ChargeA, DepthA, GrooveA = self.ParamExtract(self.DAT1)
        resnumB, self.CoordB, ChargeB, DepthB, GrooveB = self.ParamExtract(self.DAT2)

        CoordA_temp = self.CoordA
        CoordB_temp = self.CoordB
        passA1 = passA2 = passA3 = np.ones_like(resnumA)
        passB1 = passB2 = passB3 = np.ones_like(resnumB)
        # passA1 = passA2 = passA3 = np.ones(len(resnumA))
        # passB1 = passB2 = passB3 = np.ones(len(resnumB))

        if contact:
            passA1 = np.isin(resnumA, contact)
            passB1 = np.isin(resnumB, contact)
        if only_groove:
            passA2 = GrooveA == 1
            passB2 = GrooveB == 1
        if depth_cut:
            passA3 = DepthA <= depth_cut
            passB3 = DepthB <= depth_cut

        passA = passA1 * passA2 * passA3
        passB = passB1 * passB2 * passB3
        # print(passA)
        passA = passA.astype(bool)
        passB = passB.astype(bool)

        passA_index = np.where(passA == True)[0]
        CoordA_temp = CoordA_temp[passA]
        ChargeA = ChargeA[passA].reshape((-1,1))
        DepthA = DepthA[passA].reshape((-1,1))

        passB_index = np.where(passB == True)[0]
        CoordB_temp = CoordB_temp[passB]
        ChargeB = ChargeB[passB].reshape((-1,1))
        DepthB = DepthB[passB].reshape((-1,1))

        print("passA_index:", passB_index.shape)
        print("CoordA_temp:", CoordA_temp.shape)
        print("ChargeA:", ChargeA.shape)
        # Similarity score of two alleles

        SimilarityAB = self.CloudSimilarity(CoordA_temp, ChargeA, DepthA, CoordB_temp, ChargeB, DepthB)
        # SimilarityAA = self.CloudSimilarity(CoordA_temp, ChargeA, DepthA, CoordA_temp, ChargeA, DepthA)
        # SimilarityBB = self.CloudSimilarity(CoordB_temp, ChargeB, DepthB, CoordB_temp, ChargeB, DepthB)

        # print(SimilarityAB.shape)
        # print(SimilarityAA.shape)
        # print(SimilarityBB.shape)

        if (contact or only_groove or depth_cut):

            # self.score1 = pd.DataFrame(np.sum(SimilarityAA, axis=1) - 2 * np.sum(SimilarityAB, axis=1), index=passA_index)
            # self.score1 = self.score1.reindex(list(range(0,len(self.CoordA))), fill_value=0).values.reshape(-1)

            # # print(self.score1.shape)
            # self.score2 = pd.DataFrame(np.sum(SimilarityBB, axis=0) - 2 * np.sum(SimilarityAB, axis=0), index=passB_index)
            # self.score2 = self.score2.reindex(list(range(0,len(self.CoordB))), fill_value=0).values.reshape(-1)

            self.score1 = pd.DataFrame(np.sum(SimilarityAB, axis=1), index=passA_index)
            self.score1 = self.score1.reindex(list(range(0,len(self.CoordA))), fill_value=0).values.reshape(-1)

            self.score2 = pd.DataFrame(np.sum(SimilarityAB, axis=0), index=passB_index)
            self.score2 = self.score2.reindex(list(range(0,len(self.CoordB))), fill_value=0).values.reshape(-1)

        else:
            # self.score1 = pd.DataFrame(np.sum(SimilarityAA, axis=1) - 2 * np.sum(SimilarityAB, axis=1))
            # self.score2 = pd.DataFrame(np.sum(SimilarityBB, axis=0) - 2 * np.sum(SimilarityAB, axis=0))

            self.score1 = pd.DataFrame(np.sum(np.sum(SimilarityAB, axis=1)))
            self.score2 = pd.DataFrame(np.sum(np.sum(SimilarityAB, axis=0)))

        # print(self.score1.shape)
        # print(self.score2.shape)

        return

    def SavePDB(self, PDB1, PDB2, suffix="_Diff"):
        pdb1 = PandasPdb().read_pdb(PDB1)
        pdb2 = PandasPdb().read_pdb(PDB2)
        # print(print(self.score1.shape))
        # print(pdb1.df['ATOM'].shape)

        pdb1.df['ATOM']['b_factor'] = self.score1
        pdb2.df['ATOM']['b_factor'] = self.score2

        pdb1.to_pdb(Path(PDB1).stem+suffix+".pdb")
        pdb2.to_pdb(Path(PDB2).stem+suffix+".pdb")

        return

    def Show(self):
        P1 = o3d.geometry.PointCloud()
        P2 = o3d.geometry.PointCloud()

        # convert similarity score to rgb with given color map

        cmap = cm.ScalarMappable(cmap="coolwarm")

        color1 = cmap.to_rgba(self.score1)[:,0:3]
        color2 = cmap.to_rgba(self.score2)[:,0:3]
        
        # print(color1.shape)
        # print(self.CoordA.shape)
        P1.points = o3d.utility.Vector3dVector(self.CoordA)
        P1.colors = o3d.utility.Vector3dVector(color1)
        
        P2.points = o3d.utility.Vector3dVector(self.CoordB)
        P2.colors = o3d.utility.Vector3dVector(color2)

        o3d.visualization.draw_geometries([P1])
        o3d.visualization.draw_geometries([P2])
        return

In [70]:
def main():

    # DAT_dir = "HLAA_reference_panel/DAT"
    # PDB_dir = "HLAA_reference_panel/PDB"

    DAT_dir = "HLAA_relax/DAT"
    PDB_dir = "HLAA_relax/ALIGN"
    Allele1 = "A68_02"
    Allele2 = "A02_17"
    contact = [7,9,24,45,59,62,63,66,67,69,70,73,74,76,77,80,81,84,95,97,99,114,116,118,143,147,150,152,156,158,159,163,167,171]
    # DAT_dir = "HLAB_reference_panel/DAT"
    # PDB_dir = "HLAB_reference_panel/PDB"
    # Allele1 = "B07_02"
    # Allele2 = "B07_03"


    # print(f"{DAT_dir}/{Allele2}.csv")
    project = compare(f"{DAT_dir}/{Allele1}.csv", f"{DAT_dir}/{Allele2}.csv")
    project.CalcDist(depth_cut=0, contact=contact)
    # project.color_by_depth()
    # project.color_by_InGroove()

    # cmap = cm.ScalarMappable(cmap="coolwarm")
    # print(cmap.to_rgba(project.score1))

    # project.Show()
    # project.SavePDB(f"{PDB_dir}/{Allele1}.pdb", f"{PDB_dir}/{Allele2}.pdb", suffix="_depth")
    project.SavePDB(f"{PDB_dir}/{Allele1}.pdb", f"{PDB_dir}/{Allele2}.pdb", suffix="_diff")

    return

main()

passA_index: (303,)
CoordA_temp: (316, 3)
ChargeA: (316, 1)
